In [1]:
import pandas as pd
import numpy as np


In [39]:
df_campaigns = pd.read_csv("../data/raw/dim_campaigns.csv")
print(df_campaigns.shape)
df_campaigns.head()

(2, 4)


,campaign_id,campaign_name,start_date,end_date
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023
1,CAMP_SAN_01,Sankranti,10-01-2024,16-01-2024


In [40]:
df_products = pd.read_csv("../data/raw/dim_products.csv")
print(df_products.shape)
df_products.head(15)



(15, 3)


,product_code,product_name,category
0,P01,Atliq_Masoor_Dal (1KG),Grocery & Staples
1,P02,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
2,P03,Atliq_Suflower_Oil (1L),Grocery & Staples
3,P04,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
4,P05,Atliq_Scrub_Sponge_For_Dishwash,Home Care
5,P06,Atliq_Fusion_Container_Set_of_3,Home Care
6,P09,Atliq_Body_Milk_Nourishing_Lotion (120ML),Personal Care
7,P10,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care
8,P13,Atliq_High_Glo_15W_LED_Bulb,Home Appliances
9,P14,Atliq_waterproof_Immersion_Rod,Home Appliances


In [41]:
df_stores = pd.read_csv("../data/raw/dim_stores.csv")
print(df_stores.shape)
df_stores.head()

(50, 2)


,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai
2,STHYD-6,Hyderabad
3,STVSK-1,Visakhapatnam
4,STCBE-3,Coimbatore


In [42]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   store_id  50 non-null     object
 1   city      50 non-null     object
dtypes: object(2)
memory usage: 932.0+ bytes


### How many cities have more than 5 stores?

In [43]:
# Filter cities with more than 5 stores

# Step 1: Create a mask of cities with more than 5 stores
mask = df_stores['city'].value_counts() > 5

# Step 2: Extract the city names where the condition is True
values_to_keep = mask[mask].index

# Step 3: Filter the original DataFrame based on these cities
filtered_df = df_stores[df_stores['city'].isin(values_to_keep)]

# Step 4: Optional — see store counts per city
print(filtered_df['city'].value_counts())

# Step 5: Print number of such cities
num_cities = filtered_df['city'].nunique()
print(f"Number of cities that have more than 5 stores: {num_cities}")


city
Bengaluru    10
Chennai       8
Hyderabad     7
Name: count, dtype: int64
Number of cities that have more than 5 stores: 3


In [44]:
df_events = pd.read_csv("../data/raw/fact_events.csv")
print(df_events.shape)
df_events.head()

(1510, 9)


,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


#### The operations team wants to ensure the integrity of the events data by removing duplicates. Check for and remove duplicate rows in the events dataframe based on store_id, campaign_id, and product_code. How many duplicate rows were removed?

In [45]:
duplicate_events = df_events.groupby(['store_id','campaign_id','product_code']).filter(lambda x : len(x) >1)
print(f"Total Duplicaticate Events {duplicate_events.shape}")

Total Duplicaticate Events (20, 9)


In [46]:
duplicate_event_count = df_events.duplicated(subset=['store_id','campaign_id','product_code'],keep=False).sum()
print(f"Duplication Event Count {duplicate_event_count}")

Duplication Event Count 20


In [47]:
df_event_unique = df_events.drop_duplicates(subset=['store_id','campaign_id','product_code'],keep='first')
df_event_unique.shape

(1500, 9)

In [48]:
number_of_event_deleted = len(df_events)- len(df_event_unique)
print(f"Number of duplicate records deleted {number_of_event_deleted}")

Number of duplicate records deleted 10


#### The sales team has identified missing values in the quantity_sold(before_promo) data. Estimate these values using the median quantity sold before the promotion How many missing values were filled, and what is the median used for imputation?

In [49]:
df_event_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1509
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   event_id                     1500 non-null   object 
 1   store_id                     1500 non-null   object 
 2   campaign_id                  1500 non-null   object 
 3   product_code                 1500 non-null   object 
 4   base_price(before_promo)     1500 non-null   int64  
 5   quantity_sold(before_promo)  1480 non-null   float64
 6   promo_type                   1500 non-null   object 
 7   base_price(after_promo)      1500 non-null   int64  
 8   quantity_sold(after_promo)   1500 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 117.2+ KB


In [50]:
# Checking number of null values in the DataSet
df_event_unique.isnull().sum()

event_id                        0
store_id                        0
campaign_id                     0
product_code                    0
base_price(before_promo)        0
quantity_sold(before_promo)    20
promo_type                      0
base_price(after_promo)         0
quantity_sold(after_promo)      0
dtype: int64

In [51]:
nan_rows_quantity_before_promo = df_event_unique['quantity_sold(before_promo)'].isnull()
print(f"Number of rows of na values for Quantity Before Promoation: {nan_rows_quantity_before_promo.sum()}")
df_event_unique[nan_rows_quantity_before_promo]

Number of rows of na values for Quantity Before Promoation: 20


,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
63,70c312,STHYD-4,CAMP_SAN_01,P13,350,NaN,BOGOF,175,534
119,d31787,STMYS-2,CAMP_SAN_01,P10,50,NaN,25% OFF,37,20
141,141d98,STCHE-4,CAMP_SAN_01,P03,200,NaN,BOGOF,100,1695
163,873333,STMLR-0,CAMP_DIW_01,P15,3000,NaN,500 Cashback,2500,509
341,2ef46d,STMDU-0,CAMP_DIW_01,P02,860,NaN,33% OFF,576,430
391,5372de,STCBE-3,CAMP_SAN_01,P10,50,NaN,25% OFF,37,22
558,77435f,STBLR-3,CAMP_SAN_01,P01,172,NaN,33% OFF,115,387
714,a1ef43,STTRV-1,CAMP_SAN_01,P12,62,NaN,50% OFF,31,38
745,95f061,STMYS-3,CAMP_SAN_01,P15,3000,NaN,500 Cashback,2500,443
758,5f313a,STCHE-2,CAMP_DIW_01,P10,65,NaN,50% OFF,32,135


In [52]:
df_event_unique['quantity_sold(before_promo)'] = (df_event_unique['quantity_sold(before_promo)'].fillna(df_event_unique['quantity_sold(before_promo)'].median()))


/var/folders/j7/cjg4ppcx069_q5g82f26rgvh0000gn/T/ipykernel_2064/60423931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_event_unique['quantity_sold(before_promo)'] = (df_event_unique['quantity_sold(before_promo)'].fillna(df_event_unique['quantity_sold(before_promo)'].median()))


In [77]:
df_event_unique.iloc[[63,118]]

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
63,70c312,STHYD-4,CAMP_SAN_01,P13,350,78.0,BOGOF,175,534
119,d31787,STMYS-2,CAMP_SAN_01,P10,50,78.0,25% OFF,37,20


#### Identify the product category with the lowest base price before the promotion

In [54]:
df_event_unique['base_price(before_promo)'].describe()

count    1500.000000
mean      551.966667
std       741.394953
min        50.000000
25%       110.000000
50%       295.000000
75%       860.000000
max      3000.000000
Name: base_price(before_promo), dtype: float64

In [55]:
lowest_price_product_before_promotion = df_event_unique['base_price(before_promo)'].describe().min()
print(f"Lowest Price before the promotion: {lowest_price_product_before_promotion} ")

Lowest Price before the promotion: 50.0 


#### What is the total quantity sold after the promotion for the BOGOF promo type during the Diwali campaign

In [19]:
df_event_unique['promo_type'].unique()


array(['25% OFF', 'BOGOF', '33% OFF', '500 Cashback', '50% OFF'],
      dtype=object)

In [56]:
df_campaign_event_merge = pd.merge(df_campaigns, df_event_unique, on='campaign_id',how='inner')
df_campaign_event_merge.head()

,campaign_id,campaign_name,start_date,end_date,event_id,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,8481be,STCHE-1,P04,290,327.0,25% OFF,217,287
1,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,f30579,STBLR-9,P02,860,337.0,33% OFF,576,488
2,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,4f570c,STBLR-7,P05,55,122.0,25% OFF,41,107
3,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,87564a,STCHE-6,P04,290,346.0,25% OFF,217,269
4,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,6bbadf,STHYD-1,P14,1020,43.0,BOGOF,510,127


In [57]:
df_campaign_event_merge_filtered = df_campaign_event_merge[(df_campaign_event_merge['campaign_name'] == 'Diwali') & (df_campaign_event_merge['promo_type'] == 'BOGOF')]
df_campaign_event_merge_filtered.head()

,campaign_id,campaign_name,start_date,end_date,event_id,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
4,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,6bbadf,STHYD-1,P14,1020,43.0,BOGOF,510,127
5,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,66c422,STBLR-5,P07,300,66.0,BOGOF,150,227
6,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,108d5a,STCBE-0,P14,1020,29.0,BOGOF,510,101
7,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,e7d681,STCHE-5,P13,350,66.0,BOGOF,175,226
12,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,ba86f4,STVSK-1,P13,350,61.0,BOGOF,175,204


In [58]:
total_qty_sold_after_bogof_diwali = df_campaign_event_merge_filtered['quantity_sold(after_promo)'].sum()
print(f"total quantity sold after the promotion for the BOGOF promo type during the Diwali campaign: {total_qty_sold_after_bogof_diwali}")

total quantity sold after the promotion for the BOGOF promo type during the Diwali campaign: 34461


##### Which store recorded the highest quantity sold after the promotion during the Diwali campaign?

In [59]:
df_store_camp_event_merge = df_campaign_event_merge.merge(df_stores, on='store_id',how='inner')
df_store_camp_event_merge.head()

,campaign_id,campaign_name,start_date,end_date,event_id,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,8481be,STCHE-1,P04,290,327.0,25% OFF,217,287,Chennai
1,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,f30579,STBLR-9,P02,860,337.0,33% OFF,576,488,Bengaluru
2,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,4f570c,STBLR-7,P05,55,122.0,25% OFF,41,107,Bengaluru
3,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,87564a,STCHE-6,P04,290,346.0,25% OFF,217,269,Chennai
4,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,6bbadf,STHYD-1,P14,1020,43.0,BOGOF,510,127,Hyderabad


In [60]:
df_store_camp_event_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   campaign_id                  1500 non-null   object 
 1   campaign_name                1500 non-null   object 
 2   start_date                   1500 non-null   object 
 3   end_date                     1500 non-null   object 
 4   event_id                     1500 non-null   object 
 5   store_id                     1500 non-null   object 
 6   product_code                 1500 non-null   object 
 7   base_price(before_promo)     1500 non-null   int64  
 8   quantity_sold(before_promo)  1500 non-null   float64
 9   promo_type                   1500 non-null   object 
 10  base_price(after_promo)      1500 non-null   int64  
 11  quantity_sold(after_promo)   1500 non-null   int64  
 12  city                         1500 non-null   object 
dtypes: float64(1), int

In [61]:
df_store_camp_event_merge['campaign_name'].unique()

array(['Diwali', 'Sankranti'], dtype=object)

In [62]:
total_quantity_by_store = df_store_camp_event_merge.groupby('store_id')['quantity_sold(after_promo)'].sum().reset_index()
total_quantity_by_store.shape

(50, 2)

In [63]:
total_quantity_by_store.sort_values(by='quantity_sold(after_promo)',ascending=False)

,store_id,quantity_sold(after_promo)
7,STBLR-7,11866
38,STMYS-1,11773
6,STBLR-6,11601
19,STCHE-4,11547
22,STCHE-7,11546
0,STBLR-0,11425
18,STCHE-3,10951
25,STHYD-2,10822
3,STBLR-3,10752
21,STCHE-6,10731


In [64]:
top_store_by_quantity = total_quantity_by_store.nlargest(1,'quantity_sold(after_promo)')
top_store_by_quantity

,store_id,quantity_sold(after_promo)
7,STBLR-7,11866


In [65]:
print(f"The highest quantity sold after the promotion during the Diwali campaign store is: {top_store_by_quantity.store_id}  and quantiy: {top_store_by_quantity['quantity_sold(after_promo)']}")

The highest quantity sold after the promotion during the Diwali campaign store is: 7    STBLR-7
Name: store_id, dtype: object  and quantiy: 7    11866
Name: quantity_sold(after_promo), dtype: int64


##### Understand which campaigns had the most successful outcomes. Compare the total quantities sold before and after the promotions for the Sankranti and Diwali campaigns. Which campaign saw a greater increase in sales?

In [66]:
total_qty_sold_before_diwali = df_store_camp_event_merge[(df_store_camp_event_merge['campaign_name'] == 'Diwali')]['quantity_sold(before_promo)'].sum()
print(f"Total Qty Sold before Diwal Campain; {total_qty_sold_before_diwali}")
total_qty_sold_diwali = df_store_camp_event_merge[(df_store_camp_event_merge['campaign_name'] == 'Diwali')]['quantity_sold(after_promo)'].sum()
print(f"Total Qty sold after Diwali Campain: {total_qty_sold_diwali}")
total_qty_diff_diwali_camp = total_qty_sold_diwali - total_qty_sold_before_diwali 
print(f"Total Qty sold more with Diwali Camp: {total_qty_diff_diwali_camp}")

Total Qty Sold before Diwal Campain; 109756.0
Total Qty sold after Diwali Campain: 183404
Total Qty sold more with Diwali Camp: 73648.0


In [67]:
total_qty_sold_before_sankranti = df_store_camp_event_merge[(df_store_camp_event_merge['campaign_name'] == 'Sankranti')]['quantity_sold(before_promo)'].sum()
print(f"Total Qty Sold before Sankranti Campain; {total_qty_sold_before_sankranti}")
total_qty_sold_sankranti = df_store_camp_event_merge[(df_store_camp_event_merge['campaign_name'] == 'Sankranti')]['quantity_sold(after_promo)'].sum()
print(f"Total Qty Sold After Sankranti Campain; {total_qty_sold_sankranti}")

total_qty_diff_sankranti_camp = total_qty_sold_sankranti - total_qty_sold_before_sankranti 
print(f"Total Qty sold more with Sankranti Camp: {total_qty_diff_sankranti_camp}")

Total Qty Sold before Sankranti Campain; 97894.0
Total Qty Sold After Sankranti Campain; 252069
Total Qty sold more with Sankranti Camp: 154175.0


In [68]:
which_campain_more_sale = 'Sankranti ' if (total_qty_diff_sankranti_camp > total_qty_diff_diwali_camp ) else 'Diwali'
print(f"  Which campaign saw a greater increase in sales: {which_campain_more_sale}")

  Which campaign saw a greater increase in sales: Sankranti 


#### Which product recorded the highest Incremental Revenue Percentage (IR%) during the Sankranti campaign? What is the IR% for this product?

In [69]:
df_store_camp_event_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   campaign_id                  1500 non-null   object 
 1   campaign_name                1500 non-null   object 
 2   start_date                   1500 non-null   object 
 3   end_date                     1500 non-null   object 
 4   event_id                     1500 non-null   object 
 5   store_id                     1500 non-null   object 
 6   product_code                 1500 non-null   object 
 7   base_price(before_promo)     1500 non-null   int64  
 8   quantity_sold(before_promo)  1500 non-null   float64
 9   promo_type                   1500 non-null   object 
 10  base_price(after_promo)      1500 non-null   int64  
 11  quantity_sold(after_promo)   1500 non-null   int64  
 12  city                         1500 non-null   object 
dtypes: float64(1), int

In [70]:
df_store_camp_event_merge['rev_before_promo'] = df_store_camp_event_merge['base_price(before_promo)'] * df_store_camp_event_merge['quantity_sold(before_promo)']
df_store_camp_event_merge['rev_after_promo']= df_store_camp_event_merge['base_price(after_promo)'] * df_store_camp_event_merge['quantity_sold(after_promo)']

In [71]:
df_store_camp_event_merge.head()

,campaign_id,campaign_name,start_date,end_date,event_id,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city,rev_before_promo,rev_after_promo
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,8481be,STCHE-1,P04,290,327.0,25% OFF,217,287,Chennai,94830.0,62279
1,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,f30579,STBLR-9,P02,860,337.0,33% OFF,576,488,Bengaluru,289820.0,281088
2,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,4f570c,STBLR-7,P05,55,122.0,25% OFF,41,107,Bengaluru,6710.0,4387
3,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,87564a,STCHE-6,P04,290,346.0,25% OFF,217,269,Chennai,100340.0,58373
4,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,6bbadf,STHYD-1,P14,1020,43.0,BOGOF,510,127,Hyderabad,43860.0,64770


In [72]:
# Step 1: Calculate IR%
df_store_camp_event_merge['IR%'] = (
    (df_store_camp_event_merge['rev_after_promo'] - df_store_camp_event_merge['rev_before_promo'])
    / df_store_camp_event_merge['rev_before_promo']
) * 100

df_merge = df_store_camp_event_merge.merge(df_products,on='product_code')

# Step 2: Filter only Sankranti campaign rows
df_sankranti = df_merge[df_merge['campaign_name'] == 'Sankranti']

# Step 3: Group by product and get average (or total) IR%
product_ir = df_sankranti.groupby('product_name', as_index=False)['IR%'].mean()

# Step 4: Find the product with the highest IR%
top_product = product_ir.loc[product_ir['IR%'].idxmax()]

# Step 5: Display
print(f"Product with highest IR% during Sankranti: {top_product['product_name']}")
print(f"IR%: {top_product['IR%']:.2f}%")


Product with highest IR% during Sankranti: Atliq_Suflower_Oil (1L)
IR%: 104.81%


In [74]:
df_merge['city'].unique()

array(['Chennai', 'Bengaluru', 'Hyderabad', 'Coimbatore', 'Visakhapatnam',
       'Mangalore', 'Madurai', 'Mysuru', 'Vijayawada', 'Trivandrum'],
      dtype=object)

##### Which store in Visakhapatnam recorded the lowest Incremental Sold Units Percentage (ISU%) during the Diwali campaign? What is the ISU% for that store?

In [75]:
# Step 1: Calculate ISU%
df_merge['ISU%'] = (
    (df_merge['quantity_sold(after_promo)'] - df_merge['quantity_sold(before_promo)'])
    / df_merge['quantity_sold(before_promo)']
) * 100

# Step 2: Filter for Diwali campaign in Visakhapatnam
df_diwali_vizag = df_merge[
    (df_merge['campaign_name'] == 'Diwali') &
    (df_merge['city'] == 'Visakhapatnam')
]

# Step 3: Group by store and calculate average (or total) ISU%
store_isu = df_diwali_vizag.groupby('store_id', as_index=False)['ISU%'].mean()

# Step 4: Find the store with the lowest ISU%
lowest_store = store_isu.loc[store_isu['ISU%'].idxmin()]

# Step 5: Display
print(f"Store in Visakhapatnam with lowest ISU% during Diwali: {lowest_store['store_id']}")
print(f"ISU%: {lowest_store['ISU%']:.2f}%")


Store in Visakhapatnam with lowest ISU% during Diwali: STVSK-3
ISU%: 69.18%


##### Which promo type had both a negative Incremental Revenue Percentage (IR%) and Incremental Sold Units Percentage (ISU%) during the Sankranti campaign?

In [76]:
# Step 1: Calculate IR%
df_merge['IR%'] = (
    (df_merge['rev_after_promo'] - df_merge['rev_before_promo'])
    / df_merge['rev_before_promo']
) * 100

# Step 2: Calculate ISU%
df_merge['ISU%'] = (
    (df_merge['quantity_sold(after_promo)'] - df_merge['quantity_sold(before_promo)'])
    / df_merge['quantity_sold(before_promo)']
) * 100

# Step 3: Filter only Sankranti campaign
df_sankranti = df_merge[df_merge['campaign_name'] == 'Sankranti']

# Step 4: Group by promo_type and calculate average IR% and ISU%
promo_stats = (
    df_sankranti.groupby('promo_type', as_index=False)
    .agg({'IR%': 'mean', 'ISU%': 'mean'})
)

# Step 5: Filter for both negative IR% and ISU%
negative_promo = promo_stats[(promo_stats['IR%'] < 0) & (promo_stats['ISU%'] < 0)]

print("Promo types with negative IR% and ISU% during Sankranti:")
print(negative_promo)



Promo types with negative IR% and ISU% during Sankranti:
  promo_type        IR%       ISU%
0    25% OFF -39.795561 -19.172365


In [79]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   campaign_id                  1500 non-null   object 
 1   campaign_name                1500 non-null   object 
 2   start_date                   1500 non-null   object 
 3   end_date                     1500 non-null   object 
 4   event_id                     1500 non-null   object 
 5   store_id                     1500 non-null   object 
 6   product_code                 1500 non-null   object 
 7   base_price(before_promo)     1500 non-null   int64  
 8   quantity_sold(before_promo)  1500 non-null   float64
 9   promo_type                   1500 non-null   object 
 10  base_price(after_promo)      1500 non-null   int64  
 11  quantity_sold(after_promo)   1500 non-null   int64  
 12  city                         1500 non-null   object 
 13  rev_before_promo  

In [80]:
df_merge.head()

,campaign_id,campaign_name,start_date,end_date,event_id,store_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),city,rev_before_promo,rev_after_promo,IR%,product_name,category,ISU%
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,8481be,STCHE-1,P04,290,327.0,25% OFF,217,287,Chennai,94830.0,62279,-34.325635,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,-12.232416
1,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,f30579,STBLR-9,P02,860,337.0,33% OFF,576,488,Bengaluru,289820.0,281088,-3.012905,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,44.807122
2,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,4f570c,STBLR-7,P05,55,122.0,25% OFF,41,107,Bengaluru,6710.0,4387,-34.619970,Atliq_Scrub_Sponge_For_Dishwash,Home Care,-12.295082
3,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,87564a,STCHE-6,P04,290,346.0,25% OFF,217,269,Chennai,100340.0,58373,-41.824796,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,-22.254335
4,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023,6bbadf,STHYD-1,P14,1020,43.0,BOGOF,510,127,Hyderabad,43860.0,64770,47.674419,Atliq_waterproof_Immersion_Rod,Home Appliances,195.348837


##### Identify the product category with the lowest base price before the promotion

In [83]:
# Group by category and find the minimum base price before promo
lowest_price_category = (
    df_merge.groupby('category', as_index=False)['base_price(before_promo)']
    .min()
    .sort_values('base_price(before_promo)')
    .head(1)   # Get the lowest
)

print(f"Category with the lowest base price before promo: {lowest_price_category['category'].iloc[0]}")
print(f"Base Price: {lowest_price_category['base_price(before_promo)'].iloc[0]}")

Category with the lowest base price before promo: Personal Care
Base Price: 50


In [84]:
# Step 1: Filter for Diwali campaign
df_diwali = df_merge[df_merge['campaign_name'] == 'Diwali']

# Step 2: Group by store and sum quantity sold after promo
store_sales = (
    df_diwali.groupby('store_id', as_index=False)['quantity_sold(after_promo)']
    .sum()
)

# Step 3: Find the store with the highest sales
top_store = store_sales.loc[store_sales['quantity_sold(after_promo)'].idxmax()]

# Step 4: Display result
print(f"Store with highest quantity sold after promo in Diwali: {top_store['store_id']}")
print(f"Quantity Sold After Promo: {top_store['quantity_sold(after_promo)']}")


Store with highest quantity sold after promo in Diwali: STCHE-4
Quantity Sold After Promo: 5013


In [85]:
# Step 1: Group by campaign and sum quantities
campaign_sales = (
    df_merge[df_merge['campaign_name'].isin(['Sankranti', 'Diwali'])]
    .groupby('campaign_name', as_index=False)
    .agg({
        'quantity_sold(before_promo)': 'sum',
        'quantity_sold(after_promo)': 'sum'
    })
)

# Step 2: Calculate sales increase
campaign_sales['sales_increase'] = (
    campaign_sales['quantity_sold(after_promo)'] - campaign_sales['quantity_sold(before_promo)']
)

# Step 3: Find campaign with the highest increase
best_campaign = campaign_sales.loc[campaign_sales['sales_increase'].idxmax()]

# Step 4: Output in required format
print(f"{best_campaign['campaign_name']},{best_campaign['sales_increase']}")


Sankranti,154175.0


In [87]:

df_merge2 = df_merge.copy()

# Ensure numeric types
for c in ['base_price(before_promo)', 'quantity_sold(before_promo)',
          'base_price(after_promo)',  'quantity_sold(after_promo)']:
    df_merge2[c] = pd.to_numeric(df_merge2[c], errors='coerce')

# Clean product names (trailing spaces, etc.)
df_merge2['product_name'] = df_merge2['product_name'].astype(str).str.strip()

# Recompute revenues to be safe
df_merge2['rev_before_promo'] = df_merge2['base_price(before_promo)'] * df_merge2['quantity_sold(before_promo)']
df_merge2['rev_after_promo']  = df_merge2['base_price(after_promo)']  * df_merge2['quantity_sold(after_promo)']

# Filter Sankranti
sank = df_merge2[df_merge2['campaign_name'].eq('Sankranti')].copy()

# Aggregate by product (totals)
agg = (
    sank.groupby('product_name', as_index=False)
        .agg(rev_before=('rev_before_promo','sum'),
             rev_after=('rev_after_promo','sum'))
)

# Avoid divide-by-zero and bad values
agg = agg[agg['rev_before'] > 0]
agg['IR%'] = (agg['rev_after'] - agg['rev_before']) / agg['rev_before'] * 100
agg = agg.replace([np.inf, -np.inf], np.nan).dropna(subset=['IR%'])

# Top product by IR%
top = agg.loc[agg['IR%'].idxmax()]

# Required format: product_name,IR%
print(f"{top['product_name']},{top['IR%']:.2f}")


Atliq_Suflower_Oil (1L),91.83


In [88]:



df_merge3 = df_merge.copy()

# Ensure numeric columns
for c in ['quantity_sold(before_promo)', 'quantity_sold(after_promo)']:
    df_merge3[c] = pd.to_numeric(df_merge3[c], errors='coerce')

# Filter for Diwali campaign in Visakhapatnam
df_diwali_vizag = df_merge3[
    (df_merge3['campaign_name'] == 'Diwali') &
    (df_merge3['city'] == 'Visakhapatnam')
].copy()

# Aggregate total quantities per store
agg = (
    df_diwali_vizag.groupby('store_id', as_index=False)
        .agg(qty_before=('quantity_sold(before_promo)', 'sum'),
             qty_after=('quantity_sold(after_promo)', 'sum'))
)

# Avoid divide-by-zero issues
agg = agg[agg['qty_before'] > 0]
agg['ISU%'] = ((agg['qty_after'] - agg['qty_before']) / agg['qty_before']) * 100
agg = agg.replace([np.inf, -np.inf], np.nan).dropna(subset=['ISU%'])

# Find the store with lowest ISU%
lowest_store = agg.loc[agg['ISU%'].idxmin()]

# Output in requested format
print(f"{lowest_store['store_id']},{lowest_store['ISU%']:.2f}")


STVSK-3,49.21


In [90]:


df_merge5 = df_merge.copy()

# Ensure numeric columns
for c in ['base_price(before_promo)', 'quantity_sold(before_promo)',
          'base_price(after_promo)', 'quantity_sold(after_promo)']:
    df_merge5[c] = pd.to_numeric(df_merge5[c], errors='coerce')

# Compute revenues
df_merge5['rev_before_promo'] = df_merge5['base_price(before_promo)'] * df_merge5['quantity_sold(before_promo)']
df_merge5['rev_after_promo']  = df_merge5['base_price(after_promo)']  * df_merge5['quantity_sold(after_promo)']

# Filter for Sankranti campaign
df_sankranti = df_merge5[df_merge5['campaign_name'] == 'Sankranti'].copy()

# Aggregate totals per promo type
agg = (
    df_sankranti.groupby('promo_type', as_index=False)
        .agg(
            rev_before=('rev_before_promo', 'sum'),
            rev_after=('rev_after_promo', 'sum'),
            qty_before=('quantity_sold(before_promo)', 'sum'),
            qty_after=('quantity_sold(after_promo)', 'sum')
        )
)

# Remove zero or invalid before values
agg = agg[(agg['rev_before'] > 0) & (agg['qty_before'] > 0)]

# Calculate IR% and ISU%
agg['IR%']  = ((agg['rev_after'] - agg['rev_before']) / agg['rev_before']) * 100
agg['ISU%'] = ((agg['qty_after'] - agg['qty_before']) / agg['qty_before']) * 100

# Clean infinities and NaNs
agg = agg.replace([np.inf, -np.inf], np.nan).dropna(subset=['IR%', 'ISU%'])

# Filter for both negative IR% and ISU%
negative_promo = agg[(agg['IR%'] < 0) & (agg['ISU%'] < 0)]

# Output each promo type in requested format
for _, row in negative_promo.iterrows():
    print(f"{row['promo_type']},{row['IR%']:.2f},{row['ISU%']:.2f}")


25% OFF,-39.33,-19.60
